# Линейная регрессия с Statsmodels. Тестирование гипотез

**Важно** для тестирования гипотез нужно "правильно" выбрать оценку ковариационной матрицы коэффициентов. Это делаем при подгонке модеи методом `.fit(cov_type='nonrobust')`

По умолчанию используется неробастнвя OLS-оценка $Var(\beta)=s^2(X^\top X)^{-1}$

In [ ]:
import numpy as np
import pandas as pd

import statsmodels.formula.api as smf

from statsmodels.iolib.summary2 import summary_col

In [ ]:
df = pd.read_csv('Labour.csv')

## non-robust vs robust
На датасете/датафрейме `df` подгоним регрессия `log(output)` на `log(capital), log(labour), log(wage), log(capital)^2, log(labour)^2, log(wage)^2`. 

Подгоним модель с неробастной и робастной ковариационной матрицей коэффициентов

In [ ]:
# инициализация/спецификация модели через формулу
# создаём объект класса OLS
mod = smf.ols(formula='np.log(output)~np.log(capital)+np.log(labour)+np.log(wage)+I(np.log(capital)**2)+I(np.log(labour)**2)+I(np.log(wage)**2)', data=df)

res_ols = mod.fit()
res_hc = mod.fit(cov_type='HC3')

# Выведем результаты в одной таблице добавив F-статистику на значимость регрессии и число наблюдений
summary_col(results=[res_ols, res_hc], model_names=['non-robust', 'robust'], stars=True, info_dict={'F': lambda x: x.fvalue, 'Nobs': lambda x: x.nobs})

In [ ]:
res_hc.summary(slim=True)

In [ ]:
res_ols.summary(slim=True)

## t-тест
Можно тестировать общую гипотезу $H_0:\beta_j=\theta$

Используем метод `.t_test()` в котором специфицируем для каких коэффициентов какие гипотезы тестируем

Например, тестируем две гипотезы: $H_0:\beta_{\log(capital)}=0.15$ и $H_0:\beta_{\log(labour)^2}=0$ с робастной ковариационной матрицей

In [ ]:
res_hc.t_test('np.log(capital)=0.5, I(np.log(labour) ** 2)=0')

## F-тест (линейные ограничения на коэффициенты)

Линейные ограничения на коэффициенты записываются в текстовом виде

### Пример 1

Тестируем совместную значимость квадратов регрессоров с робастной оценкой ковариационной матрицы, т.е. гипотезу
$$H_0:\beta_{\log(capital)^2}=\beta_{\log(labour)^2}=\beta_{\log(wage)^2}=0$$

In [ ]:
# робастный F- тест
res_hc.f_test('I(np.log(capital) ** 2)=I(np.log(labour) ** 2)=I(np.log(wage) ** 2)=0')

In [ ]:
# неробастный F-тест
# альтернативная спецификация гипотезы
res_ols.f_test('I(np.log(capital) ** 2)=0, I(np.log(labour) ** 2)=0, I(np.log(wage) ** 2)=0')

### Пример 2

Тестируем теперь $$H_0:\beta_{\log(capital)}+\beta_{\log(labour)}=\beta_{\log(wage)}=0$$
с робастной оценкой ковариационной матрицы

In [ ]:
res_hc.f_test('np.log(capital)+np.log(labour)+np.log(wage)=1')

In [ ]:
# неробастный F-тест
res_ols.f_test('np.log(capital)+np.log(labour)+np.log(wage)=1')